In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load Data
df = pd.read_csv("sentiment_analysis.csv")

# Text Preprocessing
def preprocess_text(text):
    return re.sub(r'[^\w\s]|_\d+', '', text.lower())

df['cleaned_text'] = df['text'].apply(preprocess_text)

# Tokenization and Padding
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_text'])
padded_sequences = pad_sequences(
    tokenizer.texts_to_sequences(df['cleaned_text']), maxlen=200, padding='post'
)

# Encode Sentiments
sentiment_map = {'positive': 2, 'negative': 0, 'neutral': 1}
y = df['sentiment'].map(sentiment_map).values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# Model Definition
model = Sequential([
    Embedding(10000, 64, input_length=200),
    LSTM(64, return_sequences=True), Dropout(0.5),
    LSTM(64), Dense(32, activation='relu'), Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.2f}")

# Predict Sentiment
def predict_sentiment(text, model, tokenizer, maxlen=200):
    seq = pad_sequences(tokenizer.texts_to_sequences([preprocess_text(text)]), maxlen=maxlen, padding='post')
    sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    return sentiment_map[np.argmax(model.predict(seq), axis=1)[0]]

# Test Prediction
print(predict_sentiment("feeding babies is fun", model, tokenizer))


Epoch 1/5
7/7 [==============================] - 10s 635ms/step - loss: 1.0977 - accuracy: 0.3609 - val_loss: 1.1022 - val_accuracy: 0.3000
Epoch 2/5
7/7 [==============================] - 3s 396ms/step - loss: 1.0886 - accuracy: 0.4110 - val_loss: 1.1126 - val_accuracy: 0.3000
Epoch 3/5
7/7 [==============================] - 3s 381ms/step - loss: 1.0831 - accuracy: 0.4261 - val_loss: 1.1220 - val_accuracy: 0.3000
Epoch 4/5
7/7 [==============================] - 2s 344ms/step - loss: 1.0937 - accuracy: 0.4110 - val_loss: 1.1294 - val_accuracy: 0.3000
Epoch 5/5
4/4 [==============================] - 0s 46ms/step - loss: 1.1241 - accuracy: 0.3000
Model Accuracy: 0.30
1/1 [==============================] - 1s 1s/step
neutral
